# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.11it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.05it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.53it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.28it/s]



...

 78%|███████▊  | 18/23 [00:04<00:00,  5.03it/s]

 83%|████████▎ | 19/23 [00:04<00:00,  4.97it/s]

 87%|████████▋ | 20/23 [00:04<00:00,  4.91it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  4.91it/s]

100%|██████████| 23/23 [00:05<00:00,  4.30it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  T. Michael Phillips and I am a professional singer, songwriter, and music producer. I have been in the music industry for over 20 years, and I have worked with some of the biggest names in the industry. My goal is to create music that inspires, uplifts, and brings people together.
As a singer, I have had the privilege of performing on some of the biggest stages in the world, including the Grand Ole Opry, the Ryman Auditorium, and the iconic Wembley Stadium. My voice has been described as "powerful," "heartfelt," and "soulful," and I have been praised
Prompt: The president of the United States is
Generated text:  the head of state and government of the United States. He is the chief executive of the federal government and has a number of key responsibilities and powers.
1. The President is the head of state and government of the United States.
2. He is the commander-in-chief of the armed forces.
3. He has the power to negotiate treaties with fo

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Piper

 Red

d

 and

...

 they

 have

 in

 the

 field

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Emily and I'm a writer and a dog mom. I'm excited to share my writing adventures with you here on this blog.
I'm a bit of a goofball and love to tell stories that make people laugh. My humor is often a bit offbeat and quirky, but I like to think it's also relatable and authentic. I'm also passionate about exploring the world around us, whether that's through travel, trying new foods, or meeting new people.
As a dog mom, I'm also passionate about all things furry and four-legged. My pup, Luna, is a constant source of inspiration and joy in my life,

Prompt: The capital of France is
Generated text:  a must-see destination for anyone who loves history, art, fashion, and food. Here are some top tips for visiting Paris:
1. Learn some French: While many Parisians speak some English, learning a few basic French phrases can go a long way in making your trip more enjoyable. Try to learn how to say "bonjour" (hello), "merci" (thank you), and "excusez-m

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Nicole

 and

 I

 am

...

,

 refined

,

 and

 used

In [6]:
llm.shutdown()